# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [12]:
# Your code here
scaler = StandardScaler()

# Scale X_train and X_test using StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ensure X_train and X_test are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train_scaled = pd.DataFrame(X_train_scaled,columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled,columns=X_test.columns)

X_train_scaled

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820
...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820


In [13]:
# Your code here
regression = LinearRegression()

# Create a LinearRegression model and fit it on scaled training data
regression.fit(X_train_scaled,y_train)

# Calculate a baseline r-squared score on training data
baseline = regression.score(X_train_scaled,y_train)
baseline

0.7868344817421309

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [14]:
# Your code here

# Set up data structure
interactions = []

# Find combinations of columns and loop over them
pairs = list(combinations(X_train_scaled.columns,2))
for (c1,c2) in pairs:
    # Make copies of X_train and X_test
    train = X_train_scaled.copy()
    test = X_test_scaled.copy()
    
    # Add interaction term to data
    train['interaction'] = train[c1] * train[c2]
    test['interaction'] = test[c1] * test[c2]
    
    # Find r-squared score (fit on training data, evaluate on test data)
    score = LinearRegression().fit(train,y_train).score(test,y_test)
    
    # Append to data structure
    interactions.append(((c1,c2),score))
    
# Sort and subset the data structure to find the top 7
top_7 = sorted(interactions,key=lambda record:record[1],reverse=True)[:7]
print('Top interactions:')
print(top_7)

Top interactions:
[(('LotArea', '1stFlrSF'), 0.7211105666140569), (('LotArea', 'TotalBsmtSF'), 0.7071649207050108), (('LotArea', 'GrLivArea'), 0.6690980823779027), (('LotArea', 'Fireplaces'), 0.6529699515652585), (('2ndFlrSF', 'TotRmsAbvGrd'), 0.647299489040519), (('OverallCond', 'TotalBsmtSF'), 0.6429019879233769), (('OverallQual', '2ndFlrSF'), 0.6422324294284367)]


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_test` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [15]:
# Your code here

# Loop over top 7 interactions
for record in top_7:
    # Extract column names from data structure
    c1,c2 = record[0]
    
    # Construct new column name
    new_name = c1+'_'+c2
    
    # Add new column to X_train and X_test
    X_train_scaled[new_name] = X_train_scaled[c1]*X_train_scaled[c2]
    X_test_scaled[new_name] = X_test_scaled[c1] * X_test_scaled[c2]

In [16]:
X_train_scaled

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
0,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
1,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
2,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
3,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
4,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,-0.532331,-0.099842,-0.509252,-0.510628,-0.897228,-0.808132,-1.353116,-2.214167,-0.274466,0.588023,0.477622,0.271823,0.720306,-0.313023,1.789339,0.260039,0.080686
817,-0.309245,-0.099842,-0.509252,0.514106,0.315353,-0.808132,-0.481708,-0.365525,0.088703,-0.994820,-0.097522,-0.158985,0.148966,0.307643,0.295392,-0.261809,0.080686
818,0.119419,0.632038,-0.509252,-0.513011,-0.899947,1.684999,0.796096,0.866903,-0.207566,0.588023,-0.107471,-0.061263,0.095069,0.070221,1.460730,0.261252,1.064983
819,-0.002718,-0.099842,1.304613,-0.889542,-1.329516,0.783758,-0.290233,-0.365525,-0.852668,-0.994820,0.003613,0.002418,0.000789,0.002704,-0.286483,-1.160508,-0.078252


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [28]:
# Your code here

# Set up data structure
polynomials = []
# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for degree in (2, 3, 4):
        
        # Make a copy of X_train and X_test
        features_train = X_train_scaled.copy().reset_index()
        features_test = X_test_scaled.copy().reset_index()
    
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        col_transformed_train = pd.DataFrame(poly.fit_transform(features_train[[col]]))
        col_transformed_test = pd.DataFrame(poly.transform(features_test[[col]]))
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        features_train = pd.concat([features_train.drop(col, axis=1), col_transformed_train], axis=1)
        features_test = pd.concat([features_test.drop(col, axis=1), col_transformed_test], axis=1)
    
        # Find r-squared score
        score = LinearRegression().fit(features_train, y_train).score(features_test, y_test)
    
        # Append to data structure
        polynomials.append((col, degree, score))
        
# Sort and subset the data structure to find the top 7
top_7_polynomials = sorted(polynomials, key=lambda record: record[-1],reverse=True)[:7]
print('Top 7 Polynomials:')
print(top_7_polynomials)

Top 7 Polynomials:
[('GrLivArea', 3, 0.828318623075073), ('OverallQual', 3, 0.8068150958879934), ('OverallQual', 4, 0.8033460977380444), ('OverallQual', 2, 0.8025874910808158), ('LotArea', 4, 0.8001257921017382), ('2ndFlrSF', 3, 0.7767123915445667), ('2ndFlrSF', 4, 0.7695172607015119)]


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [29]:
# Your code here

# Filter out duplicates
top_polynomials = pd.DataFrame(top_7_polynomials, columns=["Column", "Degree", "R^2"])
top_polynomials

top_polynomials.drop_duplicates(subset="Column", inplace=True)
top_polynomials

# Loop over remaining results
for (col, degree, _) in top_polynomials.values:
    # Create polynomial terms
    poly = PolynomialFeatures(degree, include_bias=False)
    col_transformed_train = pd.DataFrame(poly.fit_transform(X_train_scaled[[col]]),
                                        columns=poly.get_feature_names([col]))
    col_transformed_test = pd.DataFrame(poly.transform(X_test_scaled[[col]]),
                                    columns=poly.get_feature_names([col]))
    # Concat new polynomials to X_train and X_test
    X_train_scaled = pd.concat([X_train_scaled.drop(col, axis=1), col_transformed_train], axis=1)
    X_test_scaled = pd.concat([X_test_scaled.drop(col, axis=1), col_transformed_test], axis=1)

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [30]:
# Your code here
X_train_scaled.head()

,OverallCond,TotalBsmtSF,1stFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,...,OverallQual,OverallQual^2,OverallQual^3,LotArea,LotArea^2,LotArea^3,LotArea^4,2ndFlrSF,2ndFlrSF^2,2ndFlrSF^3
0,-0.509252,-0.639316,-0.804789,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,...,-0.099842,0.009968,-0.000995,-0.114710,0.013158,-0.001509,1.731453e-04,1.261552,1.591512,2.007775
1,-0.509252,0.838208,0.641608,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,...,0.632038,0.399472,0.252481,-0.176719,0.031230,-0.005519,9.752976e-04,-0.808132,0.653077,-0.527772
2,1.304613,-0.012560,-0.329000,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,...,-0.831723,0.691762,-0.575354,-0.246336,0.060682,-0.014948,3.682263e-03,-0.808132,0.653077,-0.527772
3,1.304613,-0.339045,-0.609036,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,...,-0.831723,0.691762,-0.575354,-0.378617,0.143351,-0.054275,2.054946e-02,-0.808132,0.653077,-0.527772
4,1.304613,-2.531499,-1.315922,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,...,-1.563603,2.444854,-3.822780,-0.010898,0.000119,-0.000001,1.410729e-08,0.550523,0.303075,0.166850


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and test data.

In [31]:
# Your code here
lr = LinearRegression()
lr.fit(X_train_scaled,y_train)

print('Train R^2:',lr.score(X_train_scaled,y_train))
print('Test R^2:',lr.score(X_test_scaled,y_test))

Train R^2: 0.8526309398926483
Test R^2: 0.7516461451222842


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and test $R^2$ score and how many features remain.

In [32]:
# Your code here
for n in [5, 10, 15, 20, 25]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train_scaled, y_train)
    X_rfe_test = rfe.transform(X_test_scaled)

    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)

    print("Train R^2:", lr.score(X_rfe_train, y_train))
    print("Test R^2: ", lr.score(X_rfe_test, y_test))
    print(f"Using {n} out of {X_train_scaled.shape[1]} features")
    print()

Train R^2: 0.776039994126505
Test R^2:  0.6352981725272361
Using 5 out of 10 features

Train R^2: 0.8235682746440406
Test R^2:  0.769993953920479
Using 10 out of 10 features

Train R^2: 0.8358457626644923
Test R^2:  0.8369035748713005
Using 15 out of 10 features

Train R^2: 0.8494610550826857
Test R^2:  0.7996898379311601
Using 20 out of 10 features

Train R^2: 0.852630931786299
Test R^2:  0.7517809760135736
Using 25 out of 10 features



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [35]:
# Your code here
for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)

    print("Train R^2:", lasso.score(X_train_scaled, y_train))
    print("Test R^2: ", lasso.score(X_test_scaled, y_test))
    print(f"Using {sum(abs(lasso.coef_) < 10**(-10))} out of {X_train_scaled.shape[1]} features")
    print("and an alpha of", alpha)
    print()

Train R^2: 0.8520182890222521
Test R^2:  0.785139987655089
Using 0 out of 26 features
and an alpha of 1

Train R^2: 0.8520231874902416
Test R^2:  0.7850861258026375
Using 1 out of 26 features
and an alpha of 10

Train R^2: 0.8518681285356247
Test R^2:  0.7893565776644674
Using 1 out of 26 features
and an alpha of 100

Train R^2: 0.8438506939396029
Test R^2:  0.8431847022709417
Using 9 out of 26 features
and an alpha of 1000

Train R^2: 0.7841879373533859
Test R^2:  0.7852284063755455
Using 14 out of 26 features
and an alpha of 10000



Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here
# Use LASSO with alpha of 1000 and 14 features

## Evaluate Final Model on Validation Data

### Data Preparation

At the start of this lab, we created `X_val` and `y_val`. Prepare `X_val` the same way that `X_train` and `X_test` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [ ]:
# Your code here


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + test set, then find R-Squared and MSE values for the validation set.

In [ ]:
# Your code here


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.